In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Function to load and concatenate all Excel files from a list
def load_and_concatenate_excel_files(file_list):
    dataframes = []
    for file in file_list:
        df = pd.read_excel(file)
        # Ensure 'file_name' column exists by adding it
        df['file_name'] = file.split('/')[-1]  # Extract the file name from the path
        dataframes.append(df)
    concatenated_df = pd.concat(dataframes, ignore_index=True)
    return concatenated_df

# Function to perform average calculation and plotting
def calculate_average(grouped_data, x_values):
    averages = {}
    for year in grouped_data['year'].unique():
        year_data = grouped_data[grouped_data['year'] == year]
        averages[year] = [(x, year_data[year_data[x_values] == x]['number_of_posts'].sum() / year_data[year_data[x_values] == x]['number_of_active_channels'].sum() if year_data[year_data[x_values] == x]['number_of_active_channels'].sum() > 0 else 0) for x in year_data[x_values].unique()]
    return averages

def plot_averages(grouped_data, label, x_values, x_label, title, file_suffix, xticks, xtick_labels):
    averages = calculate_average(grouped_data, x_values)
    
    # Debugging output for calculated averages
    print(f"Calculated averages for {label}:")
    for year, values in averages.items():
        print(f"Year: {year}")
        for x, avg in values:
            print(f"  {x}: {avg}")
    
    plt.figure(figsize=(14, 8))
    for year, values in averages.items():
        x_vals, avg_posts = zip(*values)
        plt.plot(x_vals, avg_posts, 'o-', label=str(year))
    plt.xticks(xticks, xtick_labels)
    plt.xlabel(x_label)
    plt.ylabel(f'Average Number of {label} per Channel')
    plt.title(f'Average Number of {label} Per {title} (All Years)')
    plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    plt.grid(True)
    plt.savefig(f'{visuals_folder}/all_years_{label}_{file_suffix}_average.jpg', bbox_inches='tight')
    plt.close()

def plot_usual(grouped_data, label, x_values, x_label, title, file_suffix, xticks, xtick_labels):
    plt.figure(figsize=(14, 8))
    for year in grouped_data['year'].unique():
        year_data = grouped_data[grouped_data['year'] == year]
        plt.plot(year_data[x_values], year_data['number_of_posts'], 'o-', label=str(year))
    plt.xticks(xticks, xtick_labels)
    plt.xlabel(x_label)
    plt.ylabel(f'Number of {label}')
    plt.title(f'Number of {label} Per {title} (All Years)')
    plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    plt.grid(True)
    plt.savefig(f'{visuals_folder}/all_years_{label}_{file_suffix}.jpg', bbox_inches='tight')
    plt.close()

def perform_analysis(data, activity, label):
    data = data.dropna(subset=[activity])
    data['year'] = data['date'].dt.year
    data['hour'] = data['date_unixtime']
    data['day'] = pd.Categorical(data['date'].dt.day_name(), categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
    data['month'] = data['date'].dt.month

    grouped_hourly = data.groupby(['year', 'hour']).agg(
        number_of_posts=('date', 'size'),
        number_of_active_channels=('file_name', 'nunique')
    ).reset_index()

    grouped_daily = data.groupby(['year', 'day']).agg(
        number_of_posts=('date', 'size'),
        number_of_active_channels=('file_name', 'nunique')
    ).reset_index().sort_values(by=['year', 'day'])

    grouped_monthly = data.groupby(['year', 'month']).agg(
        number_of_posts=('date', 'size'),
        number_of_active_channels=('file_name', 'nunique')
    ).reset_index()

    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    # Hourly analysis
    plot_usual(grouped_hourly, label, 'hour', 'Hour of the Day', 'Hour', 'hourly', range(24), range(24))
    plot_averages(grouped_hourly, label, 'hour', 'Hour of the Day', 'Hour', 'hourly', range(24), range(24))

    # Daily analysis
    plot_usual(grouped_daily, label, 'day', 'Day of the Week', 'Day of the Week', 'daily', range(len(days_of_week)), days_of_week)
    plot_averages(grouped_daily, label, 'day', 'Day of the Week', 'Day of the Week', 'daily', range(len(days_of_week)), days_of_week)

    # Monthly analysis
    plot_usual(grouped_monthly, label, 'month', 'Month', 'Month', 'monthly', range(1, 13), month_names)
    plot_averages(grouped_monthly, label, 'month', 'Month', 'Month', 'monthly', range(1, 13), month_names)

def verify_averages(data, activity):
    data = data.dropna(subset=[activity])
    data['year'] = data['date'].dt.year
    data['day'] = pd.Categorical(data['date'].dt.day_name(), categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)

    grouped_daily = data.groupby(['year', 'day']).agg(
        number_of_posts=('date', 'size'),
        number_of_active_channels=('file_name', 'nunique')
    ).reset_index().sort_values(by=['year', 'day'])

    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    for year in grouped_daily['year'].unique():
        print(f"Year: {year}")
        year_data = grouped_daily[grouped_daily['year'] == year]
        for day in days_of_week:
            day_data = year_data[year_data['day'] == day]
            num_posts = day_data['number_of_posts'].sum() if not day_data.empty else 0
            num_active_channels = day_data['number_of_active_channels'].sum() if not day_data.empty else 0
            avg_posts_per_channel = num_posts / num_active_channels if num_active_channels > 0 else 0
            print(f"Day: {day} - Number of {activity.capitalize()}s: {num_posts}, Number of Active Channels: {num_active_channels}, Average {activity.capitalize()}s per Channel: {avg_posts_per_channel}")

# Define the file lists
file_list = ['new_online420fly_messages.xlsx', 'new_DalesSmokeshop_la_messages.xlsx', 'new_gollira_growers0_messages.xlsx', 'new_verified_members_la_official_messages.xlsx', 'new_jV0TiZVAMGA1MDVk_messages.xlsx', 'new_ghettomenu_messages.xlsx', 'new_vapesgmarket_messages.xlsx', 'new_vapeo_messages.xlsx', 'new_researchchemicalsconnect_messages.xlsx', 'new_Smokeplugla_messages.xlsx', 'new_TRIPPYSHIPPYDISTROMENU_messages.xlsx', 'new_Jokesupgeneticss_messages.xlsx', 'new_nycsmokeshop_messages.xlsx', 'new_thetrippyplug_messages.xlsx', 'new_Moonrocksmarket_messages.xlsx', 'new_k2spice_spray_sheets_paper_mamba_messages.xlsx', 'new_bubbafactoryla_messages.xlsx', 'new_MrTWilld_messages.xlsx', 'new_GAS_CARTEL_messages.xlsx', 'new_cbdoildestilate_messages.xlsx', 'new_WhitebluntzGumbohub_messages.xlsx', 'new_CaliPlugsMenu_messages.xlsx', 'new_smokeshoppingdelivery_messages.xlsx', 'new_weedsupplies1grower_messages.xlsx', 'new_deepmethod_messages.xlsx', 'new_narcosberlinnn_messages.xlsx', 'new_humboldtgrowersgroup_messages.xlsx', 'new_FuIlSendPackzz_messages.xlsx', 'new_laced917_messages.xlsx', 'new_Gasserybuds_messages.xlsx', 'new_trappedupp_messages.xlsx', 'new_allpacklandsla_messages.xlsx', 'new_trippyshore_messages.xlsx', 'new_BEST EXOTIC MARIJUANA 247_messages.xlsx', 'new_cali_distro_exotics_cali_plug_messages.xlsx', 'new_ssn_724365_messages.xlsx', 'new_thegunplug316_messages.xlsx', 'new_VLFlowerShopV2_messages.xlsx', 'new_Cracksnowflakeuk_messages.xlsx', 'new_Big Stoners Shop_messages.xlsx', 'new_smokeydistrol_messages.xlsx', 'new_smokeshopusa420_messages.xlsx', 'new_chatroomis_messages.xlsx', 'new_thesmokersclubuniverse_messages.xlsx', 'new_rodstraphouse_messages.xlsx', 'new_zushiwave_messages.xlsx', 'new_budiesjames_messages.xlsx', 'new_ganja420sho_messages.xlsx', 'new_whiteruntz_calikush_moonrocks_messages.xlsx', 'new_Exocticgashouse_messages.xlsx', 'new_smokeydistrola_messages.xlsx', 'new_vapingcultures_messages.xlsx', 'new_Swipe Life_messages.xlsx', 'new_gaspackstexas_messages.xlsx', 'new_Caliimport_messages.xlsx', 'new_californiagoldenshopdistro_messages.xlsx', 'new_astroexotics1_messages.xlsx', 'new_packmartlosangeIes_messages.xlsx', 'new_Vaping Culture_messages.xlsx', 'new_exotic_zaza247_messages.xlsx', 'new_trappy_exoctics_california_messages.xlsx', 'new_RubifenAdderallRitalin_messages.xlsx', 'new_traphouseexotics420_messages.xlsx', 'new_rappersfavtrapper_messages.xlsx', 'new_Trippyexoctic_messages.xlsx', 'new_zaza_carts_cookies_poundsclub_messages.xlsx', 'new_Hazydayz710_messages.xlsx', 'new_Trippysociet_messages.xlsx', 'new_drugzi_messages.xlsx', 'new_workspace2022_messages.xlsx', 'new_realpureperuviancocaine_messages.xlsx', 'new_Gasshouse070_messages.xlsx', 'new_caliplug695carte_messages.xlsx', 'new_Vaping RockNRoll_messages.xlsx', 'new_secretcannabis_messages.xlsx', 'new_kimmytantanvape_messages.xlsx', 'new_runtzog_official_messages.xlsx', 'new_flowerdistrctla_messages.xlsx', 'new_refertogetcompensated_messages.xlsx', 'new_Sherbmoney_Runtz_messages.xlsx', 'new_gascoexotics_shop_messages.xlsx', 'new_CCNumberGenerator_messages.xlsx', 'new_Buysellcannabinoidsk2spice_messages.xlsx', 'new_Mushrooms Ecstacy Pills_messages.xlsx', 'new_trappy_exotics_caliplug420_messages.xlsx', 'new_STPPromo_messages.xlsx', 'new_gasseryonly_messages.xlsx', 'new_distro_messages.xlsx', 'new_Loud_HouseExotics_messages.xlsx', 'new_texas_packs_plugs_messages.xlsx', 'new_RuntzOGcannag_messages.xlsx', 'new_Psychedelics Dispensary_messages.xlsx', 'new_trippyplugpromo_messages.xlsx', 'new_Exoticgenetix Mike_messages.xlsx', 'new_trappyexoticstouchdowns_messages.xlsx', 'new_michigangaspacks_messages.xlsx', 'new_trippyexoctic11_messages.xlsx', 'new_gasproshop302_messages.xlsx', 'new_never_soberTHC_messages.xlsx', 'new_safemedsusa_messages.xlsx', 'new_packmartmenu_messages.xlsx', 'new_trappy_exotics_caliplug_messages.xlsx', 'new_smokersclubla_messages.xlsx', 'new_Palmdale_Frisco8_messages.xlsx', 'new_sherbmoneyUS_messages.xlsx', 'new_smokeshopvendor_messages.xlsx', 'new_Trippypalacehome_messages.xlsx', 'new_MellowGodsflavors_messages.xlsx', 'new_trappyexoticsca_messages.xlsx', 'new_plugs_messages.xlsx', 'new_GassProShops_messages.xlsx', 'new_gasmartlaa_messages.xlsx']


excluded_files = ['new_CCNumberGenerator_messages.xlsx', 'new_vapeo_messages.xlsx', 'new_Smokeplugla_messages.xlsx', 
                  'new_sherbmoneyUS_messages.xlsx', 'new_Palmdale_Frisco8_messages.xlsx', 'new_flowercitygaschat_messages.xlsx']
excluded_years = [2016, 2017, 2018, 2019, 2024]

# Load and concatenate all Excel files except the excluded ones
data = load_and_concatenate_excel_files([file for file in file_list if file not in excluded_files])

# Convert 'date' column to datetime
data['date'] = pd.to_datetime(data['date'])

# Parse 'date_unixtime' to extract the hour
data['date_unixtime'] = pd.to_datetime(data['date_unixtime'], format='%H:%M:%S').dt.hour

# Exclude data from specified years
data = data[~data['date'].dt.year.isin(excluded_years)]

# Create a folder for saving visualizations
visuals_folder = 'visuals_finale5'
os.makedirs(visuals_folder, exist_ok=True)

# Verify averages for photos
verify_averages(data, 'photo')


# Perform analysis for textual messages
perform_analysis(data, 'text', 'Textual Messages')

# Perform analysis for videos
video_data = data[data['mime_type'] == 'video/mp4']
perform_analysis(video_data, 'mime_type', 'Videos')

# Perform analysis for photos
perform_analysis(data, 'photo', 'Photos')

# Perform analysis for total posts
perform_analysis(data, 'date', 'Total Activity')

print("Analysis complete and visualizations saved in 'visuals_finale1' folder")